In [1]:
path = "/media/sf_SageShared/fromScratch/"

In [2]:
M = load(path + "matroidEncyclopediaEmpty_part1.sobj")
M2 = load(path + "matroidEncyclopediaEmpty_part2.sobj")
M.update(M2)
#loads the empty Matroid Encyclopedia, including minor relations and circuit-hyperplane relaxation relations.

In [3]:
def kDual(k):
    return (k[1]-k[0],k[1])
def kDel(k):
    return (k[0],k[1]-1)
def kCon(k):
    return (k[0]-1,k[1]-1)

def addProperty(E,A,val,certificate=False):
    if A in E:
        if not E[A] == val:
            print "error!", k, mi, A, val, E[A]
    else:
        E[A] = val
        if certificate != False:
            E[A + " certificate"] = certificate

def AimpliesB(A,B):
    for k in M:
        for mi in M[k]:
            E = M[k][mi]
            if A in E and E[A]:
                addProperty(E,B,True,A)
            elif B in E and not E[B]:
                addProperty(E,A,False,B)

                
def closeDuality(A,certificate=True):
    for k in M:
        kd = kDual(k)
        for mi in M[k]:
            E = M[k][mi]
            if not A in E:
                continue
            di = E["dual"]
            Ed = M[kd][di]
            addProperty(Ed,A,E[A],("duality" if certificate else False))

def inheritFromMinor(A,val,principal=False,deletion=True,contraction=True,certificate=True):
    minorsr = "minors r"
    minorsr1 = "minors r-1"
    if principal:
        minorsr = "principal minors r"
        minorsr1 = "principal minors r-1"
    
    for n in range(10):
        for k in M:
            if k[1] != n:
                continue
            for mi in M[k]:
                E = M[k][mi]
                if A in E:
                    continue

                if deletion:
                    kd = kDel(k)
                    for i in E[minorsr]:
                        Ed = M[kd][i]
                        if A in Ed and Ed[A] == val:
                            addProperty(E,A,val,([minorsr,i] if certificate else False))
                            break

                    if A in E:
                        continue

                if contraction:            
                    kc = kCon(k)
                    for i in E[minorsr1]:
                        Ec = M[kc][i]
                        if A in Ec and Ec[A] == val:
                            addProperty(E,A,val,([minorsr1,i] if certificate else False))
                            break
                        
                        
def inheritFromExtension(A,val,principal=False,ext=True,coext=True,certificate=True):
    minorsr = "minors r"
    minorsr1 = "minors r-1"
    certr = "extension"
    certr1 = "coextension"
    if principal:
        minorsr = "principal minors r"
        minorsr1 = "principal minors r-1"
        certr = "principal extension"
        certr1 = "principal coextension"
    ns = range(10)
    ns.reverse()
    for n in ns:
        for k in M:
            if k[1] != n:
                continue
            for mi in M[k]:
                E = M[k][mi]
                if not (A in E and E[A] == val):
                    continue

                if ext:
                    kd = kDel(k)
                    for i in E[minorsr]:
                        Ed = M[kd][i]
                        addProperty(Ed,A,val,([certr,mi] if certificate else False) )

                if coext:            
                    kc = kCon(k)
                    for i in E[minorsr1]:
                        Ec = M[kc][i]
                        addProperty(Ec,A,val,([certr1,mi] if certificate else False))
                    
                    
def closeCHPrelaxation(A,val):
    for k in M:
        nChanged = 1
        while nChanged > 0:
            nChanged = 0
            for mi in M[k]:
                E = M[k][mi]
                if not (A in E and E[A] == val):
                    continue
                for i in E["circuit-hyperplane relaxations"]:
                    Er = M[k][i]
                    if not (A in Er):
                        nChanged += 1
                        addProperty(Er,A,val,["circuit-hyperplane relaxation", mi])

                    
def rank2Linear():
    for k in M:
        if k[0] <= 2 or k[1]-k[0] <= 2:
            for mi in M[k]:
                E = M[k][mi]
                addProperty(E,"2linear",True,"(co)rank at most 2")
                
                
def inspect(A,k,onlySize=True):
    values = {"-":[]}
    for mi in M[k]:
        E = M[k][mi]
        if not A in E:
            values["-"].append(mi)
        else:
            v = str(E[A])
            if not (v in values):
                values[v] = [mi]
            else:
                values[v].append(mi)
    if onlySize:
        return {v: len(values[v]) for v in values}
    else:
        return values            

In [4]:
rank2Linear()

In [5]:
binary = load(path + "binaryMatroids.sobj")
for k in binary:
    for mi in binary[k]:
        addProperty(M[k][mi],"2linear",True,"binary")
del binary

In [6]:
lin93 = load(path + "linear2_39.sobj")
#representations found in "linear2representations93.sobj"; requires Sage 8.0 or higher
n = 9
for rep in lin93:
    r = rep[1]
    k = (r,n)
    addProperty(M[k][rep[0]],"2linear",True,"represented")
del lin93

In [7]:
lin94 = load(path + "linear2_49.sobj")
#representations found in "linear2representations94.sobj"; requires Sage 8.0 or higher
n = 9
for rep in lin94:
    r = rep[1]
    k = (r,n)
    addProperty(M[k][rep[0]],"2linear",True,"represented")
del lin94

In [8]:
nonlin94 = load(path + "non2linear49_extra.sobj")
for mi in nonlin94:
    addProperty(M[k][mi],"2linear",False,"Groebner")
del nonlin94

In [9]:
closeDuality("2linear")

In [10]:
rat93 = load(path + "rationalRepresentations39.sobj")
n = 9
for rep in rat93:
    r = rep[1].nrows()
    k = (r,n)
    addProperty(M[k][rep[0]],"rational",True,"represented")
del rat93

In [11]:
rat94 = load(path + "rationalRepresentations49.sobj")
n = 9
for rep in rat94:
    r = rep[1].nrows()
    k = (r,n)
    addProperty(M[k][rep[0]],"rational",True,"represented")
del rat94

In [12]:
closeDuality("rational")

In [13]:
nonDW = load(path + "NonDressWenzel.sobj")
reg = nonDW["regular"]
for k in reg:
    for mi in reg[k]:
        addProperty(M[k][mi],"Dress-Wenzel condition",False)
    for mi in set([i for i in M[k]]) - set(reg[k]):
        addProperty(M[k][mi],"Dress-Wenzel condition",True)
    
char2 = nonDW["char 2"]
for k in char2:
    for mi in char2[k]:
        addProperty(M[k][mi],"Dress-Wenzel condition char 2",False)
    for mi in set([i for i in M[k]]) - set(char2[k]):
        addProperty(M[k][mi],"Dress-Wenzel condition char 2",True)
del nonDW
del char2
del reg

In [14]:
closeDuality("Dress-Wenzel condition",False)

In [15]:
closeDuality("Dress-Wenzel condition char 2",False)

In [16]:
nonIM = load(path + "NonIngletonMain.sobj")
for k in nonIM:
    for d in nonIM[k]:
        for mi in nonIM[k][d]:
            addProperty(M[k][mi],"Ingleton-Main condition",False)
            addProperty(M[k][mi],"Ingleton-Main condition depth",d)
del nonIM

In [17]:
inheritFromMinor("Ingleton-Main condition",False,certificate=False)

In [18]:
addProperty(M[3,9][594],"2flock",False,"computed")
addProperty(M[6,9][594],"2flock",False,"computed")
#This is the ternary Reid geometry (and its dual)

addProperty(M[4,8][383],"2flock",False,"computed")
#This is the matroid T8.

addProperty(M[4,8][259],"2flock",False,"computed")

addProperty(M[3,9][560],"Skew-field representable",True,"represented (Lindstrom)")
#This is the Non-Pappus matroid

addProperty(M[3,9][655],"Quaternion representable",True,"represented")
#This is the matroid from Example 8.4

In [19]:
quat = load(path + "QuaternionRepresentable49.sobj")
for mi in quat:
    addProperty(M[4,9][mi],"Quaternion representable",True,"represented")
del quat

In [20]:
closeDuality("Quaternion representable")
#these are all over a commutative subfield of the quaternions, and thus may be closed under duality

In [21]:
numb = load(path + "NumberFieldRepresentable49.sobj")
for mi in numb:
    addProperty(M[4,9][mi],"Quadratic imaginary number field representable",True,"represented")
del numb

In [22]:
closeDuality("Quadratic imaginary number field representable")

In [23]:
grid = load(path + "LindstromGridMatroids49.sobj")
for mi in grid:
    addProperty(M[4,9][mi],"Skew-field representable",True,"represented (Lindstrom)")
del grid

In [24]:
closeDuality("Skew-field representable")

In [25]:
flock = load(path + "FlockRepresentable.sobj")
for k in flock:
    for mi in flock[k]:
        addProperty(M[k][mi],"2flock",True,"computed")
del flock

In [26]:
nonflock = load(path + "nonFlock49.sobj")
for mi in nonflock:
    addProperty(M[4,9][mi],"2flock",False,"computed")
    addProperty(M[5,9][mi],"2flock",False,"computed")
del nonflock

In [27]:
AimpliesB("2linear","Dress-Wenzel condition char 2")

In [28]:
AimpliesB("rational","Dress-Wenzel condition")

In [29]:
AimpliesB("Quadratic imaginary number field representable","Dress-Wenzel condition")

In [30]:
inheritFromExtension("2linear",True)
inheritFromExtension("rational",True)
inheritFromExtension("Quaternion representable",True)
inheritFromExtension("Skew-field representable",True)
inheritFromExtension("Quadratic imaginary number field representable",True)

In [31]:
inheritFromMinor("Skew-field representable",True,principal=True,contraction=False)
closeDuality("Skew-field representable")

In [32]:
AimpliesB("2linear","2algebraic")

In [33]:
AimpliesB("rational","2algebraic")

In [34]:
AimpliesB("Quaternion representable","2algebraic")

In [35]:
AimpliesB("Quadratic imaginary number field representable","2algebraic")

In [36]:
AimpliesB("Skew-field representable","2algebraic")

In [37]:
AimpliesB("2algebraic","Ingleton-Main condition")

In [38]:
inheritFromMinor("2algebraic",True,principal=True,contraction=False)
#principal extension is currently not required to fill the encyclopedia.

In [39]:
AimpliesB("2algebraic","2flock")

In [40]:
closeCHPrelaxation("2flock",True)

In [41]:
inspect("2algebraic",(4,9))

{'-': 10747, 'False': 31820, 'True': 147647}

In [42]:
inspect("2algebraic certificate",(4,9))

{'-': 10747,
 '2flock': 1064,
 '2linear': 125692,
 'Ingleton-Main condition': 30756,
 'Quadratic imaginary number field representable': 284,
 'Quaternion representable': 1104,
 'Skew-field representable': 98,
 'rational': 20469}

In [43]:
inspect("2algebraic",(5,9))

{'-': 13304, 'False': 29263, 'True': 147647}

In [44]:
inspect("2algebraic certificate",(5,9))

{'-': 13304,
 '2flock': 1763,
 '2linear': 125692,
 'Ingleton-Main condition': 27500,
 'Quadratic imaginary number field representable': 284,
 'Quaternion representable': 1104,
 'Skew-field representable': 98,
 'rational': 20469}

In [45]:
inspect("2algebraic",(3,9))

{'-': 3, 'False': 1, 'True': 1271}

In [46]:
inspect("2algebraic",(6,9))

{'-': 3, 'False': 1, 'True': 1271}

In [47]:
inspect("2algebraic",(4,8))

{'-': 3, 'False': 40, 'True': 897}

In [48]:
inspect("Dress-Wenzel condition",(4,9))

{'-': 0, 'False': 37988, 'True': 152226}

In [49]:
inspect("Dress-Wenzel condition char 2",(4,9))

{'-': 0, 'False': 64395, 'True': 125819}

In [50]:
inspect("Ingleton-Main condition",(4,9))

{'-': 11811, 'False': 30756, 'True': 147647}

In [51]:
inspect("Ingleton-Main condition depth",(4,9))

{'-': 186595, '3': 1281, '4': 1753, '5': 487, '6': 98}

In [52]:
inspect("Ingleton-Main condition",(5,9))

{'-': 15067, 'False': 27500, 'True': 147647}

In [53]:
inspect("Ingleton-Main condition depth",(5,9))

{'-': 189851, '3': 7, '4': 298, '5': 58}

In [54]:
inspect("2flock",(3,9))

{'-': 0, 'False': 1, 'True': 1274}

In [55]:
inspect("2flock",(4,8))

{'-': 0, 'False': 2, 'True': 938}

In [56]:
inspect("2flock",(4,9))

{'-': 317, 'False': 4551, 'True': 185346}

In [57]:
inspect("2flock",(5,9))

{'-': 315, 'False': 4551, 'True': 185348}